In [1]:
import ctypes
import numpy as np
import utility.Pi_util
import importlib
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
importlib.reload(utility.Pi_util)
path: str = '/Volumes/EXTERNAL1/newJson/'
results: dict = utility.Pi_util.parse_chunks_binary(path, use_mean_frame=False)

In [ ]:
print(results)

In [ ]:
performance_df: pd.DataFrame = results['performance_df']
chunks: list = results['chunks']

# Iterate over the chunks 
for chunk_num, chunk in enumerate(chunks):
    print(f"CHUNK: {chunk_num}")

    # Generate a plot for raw and demosaiced world images
    #fig, axes = plt.subplots(1,2)




    # Retrieve the difference sensors' data 
    #ms_data: tuple = chunk['M'] 
    world_data: np.ndarray = chunk['W']

    #for i in range(100,1000, 100):
    #    plt.imshow(world_data[i], cmap='gray')
    #    plt.show()

    #plt.show()

    print(world_data.shape)

    #pupil_data: list = chunk['P']
    #sunglasses_data: np.ndarray = chunk['S']

    #print(pupil_data.shape)

    #if(pupil_data.shape[0] == 0):
    #    continue

    plt.imshow(world_data[1000], cmap='gray')
    plt.show()

    #print(world_data.shape)
    #print(pupil_data.shape)
    


In [ ]:
print(performance_df)